## EZKL HUB Jupyter Notebook Demo 

Here we demonstrate the use of the EZKL hub in a Jupyter notebook whereby all components of the circuit are public or pre-committed to. This is the simplest case of using EZKL (proof of computation).

This will be accomplished in 3 steps. 

1. Train the model. 
2. Define our visibility settings. 
3. Upload the model to the hub. 

That's it !

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


circuit = MyModel()

# Train the model as you like here (skipped for brevity)

In [ ]:
model_path = os.path.join('network.onnx')
data_path = os.path.join('input.json')
cal_path = os.path.join('calibration.json')

In [ ]:
# After training, export to onnx (network.onnx) and create a data file (input.json)
x = 0.1*torch.rand(1,*[1, 28, 28], requires_grad=True)




# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize input data into file:
json.dump( data, open(data_path, 'w' ))

# generate calibration data using 10 random input samples

# create an array of random sample input tensors
x = 0.1*torch.rand(10,*[1, 28, 28], requires_grad=True)

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize cal data into file:
json.dump( data, open(cal_path, 'w' ))


In [ ]:
import time
test_hub_name = "samtvlabs" #we've set this up for you, but you can create your own hub name and use that instead

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "private" # private by default


organization = ezkl.get_hub_credentials(test_hub_name)['organizations'][0]

print("organization: " + str(organization))

# timestamped name
name = "model_" + str(int(time.time()))

deployed_model = ezkl.create_hub_artifact(model_path, data_path, name, organization['id'], cal_input=cal_path, target="resources", py_run_args=py_run_args)

print("deployed model: " + str(deployed_model))


In [ ]:
# Loop every 5 seconds until status is not pending
status = "PENDING"
while status == "PENDING":
    time.sleep(5)
    get_model = ezkl.get_hub_artifact(deployed_model['id'])
    status = get_model['status']
print("status: " + status)

In [ ]:

proof_id = ezkl.prove_hub(deployed_model['id'], data_path)
print("proof id: " + str(proof_id))

In [ ]:
# Loop every 5 seconds until status is not pending
status = "PENDING"
while status == "PENDING":
    time.sleep(5)
    get_proof = ezkl.get_hub_proof(proof_id['id'])
    status = get_proof['status']

In [ ]:
proof = ezkl.get_hub_proof(proof_id['id'])

print("proof: " + str(proof))